In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [3]:
%load_ext autoreload
%autoreload 2
from scripts.text.semantic_graph import SemanticGraph
from scripts.extractive_summary.key_events_graph import KeyEventsSummaryGraph

from scripts.conf import TEAMS

import pandas as pd

%reload_ext autoreload

# Extracción de eventos importantes usando grafos conceptuales

El objetivo de este experimento es utilizar los eventos de los partidos para crear un grafo conceptual, y extraer a partir de este los eventos 
más importantes (atendiendo al grado de los nodos, que serán palabras/entidades nombradas/sujetos nominales).

## Probando con eventos de un partido

In [4]:
key_events = KeyEventsSummaryGraph()

In [5]:
all_files = key_events.processor.load_json()

In [6]:
season_file = 'premier_league_2018_2019.json'
league_season_teams = TEAMS[season_file.split('.')[0]]
# Solo para pruebas
key_events.processor.league_season_teams = league_season_teams

In [7]:
match = all_files[season_file]['http://www.premierleague.com/match/38678']

In [12]:
event_list = match['events']

In [11]:
event_list

['Attempt blocked. Ilkay Gündogan (Manchester City) right footed shot from outside the box is blocked. Assisted by Bernardo Silva.',
 'Corner,  Brighton and Hove Albion. Conceded by Aymeric Laporte.',
 'Offside, Brighton and Hove Albion. Anthony Knockaert tries a through ball, but Glenn Murray is caught offside.',
 'Foul by Anthony Knockaert (Brighton and Hove Albion).',
 'Aymeric Laporte (Manchester City) wins a free kick in the defensive half.',
 'Attempt missed. Alireza Jahanbakhsh (Brighton and Hove Albion) right footed shot from outside the box is close, but misses to the left. Assisted by Anthony Knockaert following a fast break.',
 'Attempt blocked. Sergio Agüero (Manchester City) right footed shot from outside the box is blocked. Assisted by Bernardo Silva.',
 'Foul by Yves Bissouma (Brighton and Hove Albion).',
 'Bernardo Silva (Manchester City) wins a free kick in the defensive half.',
 'Attempt missed. Sergio Agüero (Manchester City) header from the centre of the box misses 

In [17]:
semantic_graph = SemanticGraph(event_list, league_season_teams)
g = semantic_graph.create_graph()

In [18]:
hubs_sentences = semantic_graph.get_n_hubs_sentences(n=10)

Hubs: [('brighton', 56), ('manchester city', 46), ('attempt', 36), ('the box', 36), ('hove', 34), ('albion', 34), ('goal', 30), ('hove albion', 29), ('the centre', 27), ('a free kick', 26)]


In [20]:
semantic_graph = SemanticGraph(event_list, league_season_teams, drop_teams=True, only_players=True)
g = semantic_graph.create_graph()

In [16]:
# Solo con jugadores
hubs_sentences = semantic_graph.get_n_hubs_sentences(n=10)

Hubs: [('sergio agüero', 5), ('riyad mahrez', 5), ('ilkay gündogan', 3), ('anthony knockaert', 3), ('glenn murray', 3), ('david silva', 3), ('bernardo silva', 2), ('alireza jahanbakhsh', 2), ('raheem sterling', 2), ('kyle walker', 2)]


In [42]:
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
n_hubs = 5
fc = 0.1
mode = 'homogeneous'

In [43]:
key_events = KeyEventsSummaryGraph(drop_teams=True, only_players=True)

In [44]:
season_file = 'premier_league_2018_2019.json'
league_season_teams = TEAMS[season_file.split('.')[0]]
# Solo para pruebas
key_events.processor.league_season_teams = league_season_teams

In [27]:
processed_events = key_events.process_match_events(event_list, n_hubs, fc, mode)

Hubs: [('sergio agüero', 5), ('riyad mahrez', 5), ('ilkay gündogan', 3), ('anthony knockaert', 3), ('glenn murray', 3), ('david silva', 3), ('bernardo silva', 2), ('alireza jahanbakhsh', 2), ('raheem sterling', 2), ('kyle walker', 2)]
Hubs with sentences: {'sergio agüero': 9, 'riyad mahrez': 8, 'ilkay gündogan': 7, 'anthony knockaert': 9, 'glenn murray': 5, 'david silva': 7, 'bernardo silva': 3, 'alireza jahanbakhsh': 3, 'raheem sterling': 6, 'kyle walker': 6}
The text has 83 events
The semantic graph has 23 nodes
The summary should have 42 sentences with a compression factor of 0.5
There are 50 sentences in the 10 nodes with more degree
Number of original events: 83
Number of processed events: 42


In [28]:
key_events.match_summary(match, count_vec_kwargs, save_relations=True, verbose=True, n_hubs=10, 
                         fc=fc, mode=mode)

Hubs: [('sergio agüero', 5), ('riyad mahrez', 5), ('ilkay gündogan', 3), ('anthony knockaert', 3), ('glenn murray', 3), ('david silva', 3), ('bernardo silva', 2), ('alireza jahanbakhsh', 2), ('raheem sterling', 2), ('kyle walker', 2)]
Hubs with sentences: {'sergio agüero': 9, 'riyad mahrez': 8, 'ilkay gündogan': 7, 'anthony knockaert': 9, 'glenn murray': 5, 'david silva': 7, 'bernardo silva': 3, 'alireza jahanbakhsh': 3, 'raheem sterling': 6, 'kyle walker': 6}
The text has 83 events
The semantic graph has 23 nodes
The summary should have 42 sentences with a compression factor of 0.5
There are 50 sentences in the 10 nodes with more degree
Number of original events: 83
Number of processed events: 42
Number of sentences in original article: 6
Number of sentences in summary: 4
Event:
Attempt blocked. Ilkay Gündogan (Manchester City) right footed shot from outside the box is blocked. Assisted by Bernardo Silva.
Nearest sentence in article:
After the break, Mahrez fired superbly into the top

{'article_summary': "A goal and an assist from Riyad Mahrez sealed a 4-1 comeback win at Brighton & Hove Albion that confirmed Manchester City as Premier League champions.\xa0\nGlenn Murray gave Brighton a shock lead when he headed in Pascal Gross's 27th-minute corner.\nBut Sergio Aguero levelled within a minute after being played in by David Silva, before Aymeric Laporte headed City in front from Mahrez's corner on 38 minutes.\nAfter the break, Mahrez fired superbly into the top corner before Ilkay Gundogan scored a brilliant free-kick with 18 minutes remaining.\n",
 'sentences_ixs': array([4, 2, 0, 0, 3, 3, 0, 0, 4, 2, 2, 0, 0, 2, 3, 3, 0, 3, 3, 0, 3, 4,
        0, 3, 3, 0, 4, 0, 3, 4, 3, 4, 0, 2, 4, 3, 0, 3, 0, 0, 3, 3]),
 'article_sents_list': ['A goal and an assist from Riyad Mahrez sealed a 4-1 comeback win at Brighton & Hove Albion that confirmed Manchester City as Premier League champions.\xa0\n',
  "The Algerian, a 2015/16 champion with Leicester City, replaced Phil Foden in t

In [30]:
' '.join(list(map(match['events'].__getitem__, key_events.events_mapping_list)))

'Attempt blocked. Ilkay Gündogan (Manchester City) right footed shot from outside the box is blocked. Assisted by Bernardo Silva. Offside, Brighton and Hove Albion. Anthony Knockaert tries a through ball, but Glenn Murray is caught offside. Foul by Anthony Knockaert (Brighton and Hove Albion). Attempt missed. Alireza Jahanbakhsh (Brighton and Hove Albion) right footed shot from outside the box is close, but misses to the left. Assisted by Anthony Knockaert following a fast break. Attempt blocked. Sergio Agüero (Manchester City) right footed shot from outside the box is blocked. Assisted by Bernardo Silva. Attempt missed. Sergio Agüero (Manchester City) header from the centre of the box misses to the left. Assisted by Raheem Sterling with a cross. Corner,  Manchester City. Conceded by Anthony Knockaert. Attempt saved. Riyad Mahrez (Manchester City) left footed shot from outside the box is saved in the bottom right corner. Assisted by Kyle Walker. Attempt blocked. Ilkay Gündogan (Manches

In [45]:
processed_events = key_events.process_match_events(event_list, n_hubs, fc, mode)

Hubs: [('sergio agüero', 5), ('riyad mahrez', 5), ('ilkay gündogan', 3), ('anthony knockaert', 3), ('glenn murray', 3)]
Hubs with sentences: {'sergio agüero': 9, 'riyad mahrez': 8, 'ilkay gündogan': 7, 'anthony knockaert': 9, 'glenn murray': 5}
The text has 83 events
The semantic graph has 23 nodes
The summary should have 8 sentences with a compression factor of 0.1
There are 35 sentences in the 5 nodes with more degree
Number of original events: 83
Number of processed events: 8


In [46]:
' '.join(list(map(match['events'].__getitem__, key_events.events_mapping_list)))

'Attempt blocked. Sergio Agüero (Manchester City) right footed shot from outside the box is blocked. Assisted by Bernardo Silva. Attempt missed. Sergio Agüero (Manchester City) header from the centre of the box misses to the left. Assisted by Raheem Sterling with a cross. Goal.  Brighton and Hove Albion 1, Manchester City 1. Sergio Agüero (Manchester City) left footed shot from a difficult angle on the left to the centre of the goal. Assisted by David Silva with a through ball. Foul by Sergio Agüero (Manchester City). Foul by Sergio Agüero (Manchester City). Attempt blocked. Ilkay Gündogan (Manchester City) right footed shot from outside the box is blocked. Assisted by Sergio Agüero. Sergio Agüero (Manchester City) wins a free kick in the attacking half. Sergio Agüero (Manchester City) wins a free kick on the left wing.'

### Con todos

In [4]:
summaries_path = '{}summaries/key_events_summaries_graph_8.csv'.format(CSV_DATA_PATH)
map_path = '{}summaries/key_events_summaries_graph_8_map.csv'.format(CSV_DATA_PATH)
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
n_hubs = 10
fc = 0.1
mode = 'heterogeneous'

In [5]:
map_path

'/home/carlos/MasterDS/tfm/data/csv/summaries/key_events_summaries_graph_6_map.csv'

In [5]:
key_events = KeyEventsSummaryGraph(drop_teams=True)

In [6]:
%%time
pd_df = key_events.run(save_events_sentences=True, path_csv=summaries_path, 
                       path_mapping=map_path, count_vec_kwargs=count_vec_kwargs,
                       n_hubs=n_hubs, fc=fc, mode=mode
                       )

  0%|          | 0/20 [00:00<?, ?it/s]

Hubs: [('attempt', 31), ('the box', 31), ('goal', 26), ('the centre', 23), ('a cross', 22), ('footed shot', 22), ('a free kick', 21), ('riyad mahrez', 20), ('sergio agüero', 19), ('ilkay gündogan', 17)]
Hubs with sentences: {'attempt': 21, 'the box': 19, 'goal': 5, 'the centre': 9, 'a cross': 5, 'footed shot': 7, 'a free kick': 21, 'riyad mahrez': 6, 'sergio agüero': 9, 'ilkay gündogan': 8}
The text has 83 events
The semantic graph has 60 nodes
The summary should have 8 sentences with a compression factor of 0.1
There are 49 sentences in the 10 nodes with more degree
Number of original events: 83
Number of processed events: 8
Number of sentences in original article: 6
Number of sentences in summary: 4
Hubs: [('attempt', 41), ('the box', 36), ('a cross', 28), ('the centre', 26), ('alex iwobi', 22), ('ashley barnes', 21), ('chris wood', 21), ('pierre-emerick aubameyang', 20), ('footed shot', 20), ('the left', 19)]
Hubs with sentences: {'attempt': 25, 'the box': 19, 'a cross': 8, 'the cen

  5%|▌         | 1/20 [16:51<5:20:16, 1011.39s/it]

Number of sentences in original article: 6
Number of sentences in summary: 2
Hubs: [('attempt', 27), ('the box', 26), ('the centre', 21), ('footed shot', 15), ('a cross', 14), ('the bottom right corner', 12), ('a free kick', 11), ('pablo aguilar', 11), ('the right', 10), ('josé luis cordero', 10)]
Hubs with sentences: {'attempt': 18, 'the box': 15, 'the centre': 8, 'footed shot': 7, 'a cross': 3, 'the bottom right corner': 5, 'a free kick': 40, 'pablo aguilar': 5, 'the right': 4, 'josé luis cordero': 6}
The text has 118 events
The semantic graph has 52 nodes
The summary should have 12 sentences with a compression factor of 0.1
There are 65 sentences in the 10 nodes with more degree
Number of original events: 118
Number of processed events: 12
Number of sentences in original article: 14
Number of sentences in summary: 6
Hubs: [('new york city fc', 42), ('attempt', 40), ('the box', 37), ('footed shot', 27), ('the centre', 26), ('david villa', 22), ('bradley wright-phillips', 22), ('a fre

/home/carlos/MasterDS/tfm/scripts/extractive_summary/key_events_graph.py:154: UserWarning: Required hubs are higher than available nodes. Using 2 hubs
  warnings.warn('Required hubs are higher than available nodes. Using {} hubs'.format(n_nodes))


...

 95%|█████████▌| 19/20 [2:54:29<03:16, 196.52s/it]

Number of sentences in original article: 1
Number of sentences in summary: 1
Hubs: [('the centre', 35), ('attempt', 33), ('the box', 33), ('ciro immobile', 26), ('footed shot', 25), ('senad lulic', 24), ('goal', 22), ('ball', 21), ('the goal', 20), ('marco parolo', 20)]
Hubs with sentences: {'the centre': 19, 'attempt': 30, 'the box': 22, 'ciro immobile': 15, 'footed shot': 9, 'senad lulic': 14, 'goal': 4, 'ball': 11, 'the goal': 7, 'marco parolo': 7}
The text has 129 events
The semantic graph has 78 nodes
The summary should have 13 sentences with a compression factor of 0.1
There are 51 sentences in the 10 nodes with more degree
Number of original events: 129
Number of processed events: 13
Number of sentences in original article: 8
Number of sentences in summary: 3
Hubs: [('the box', 34), ('attempt', 32), ('the centre', 29), ('a cross', 24), ('mario mandzukic', 19), ('cristiano ronaldo', 18), ('the left', 15), ('paulo dybala', 14), ('giorgio chiellini', 14), ('ivan perisic', 13)]
Hubs

100%|██████████| 20/20 [2:56:55<00:00, 530.77s/it]

Number of sentences in original article: 8
Number of sentences in summary: 5


Saving summaries in /home/carlos/MasterDS/tfm/data/csv/summaries/key_events_summaries_graph_8.csv
Saving mappings in /home/carlos/MasterDS/tfm/data/csv/summaries/key_events_summaries_graph_8_map.csv
CPU times: user 2h 55min 53s, sys: 1min 18s, total: 2h 57min 11s
Wall time: 2h 56min 59s


In [8]:
pd_df['summary_events'].values[0]

'Corner,  Brighton and Hove Albion. Conceded by Aymeric Laporte. Offside, Brighton and Hove Albion. Anthony Knockaert tries a through ball, but Glenn Murray is caught offside. Foul by Anthony Knockaert (Brighton and Hove Albion). Attempt missed. Alireza Jahanbakhsh (Brighton and Hove Albion) right footed shot from outside the box is close, but misses to the left. Assisted by Anthony Knockaert following a fast break. Foul by Yves Bissouma (Brighton and Hove Albion). Foul by Glenn Murray (Brighton and Hove Albion). Glenn Murray (Brighton and Hove Albion) wins a free kick in the defensive half. Attempt blocked. Alireza Jahanbakhsh (Brighton and Hove Albion) right footed shot from outside the box is blocked. Corner,  Brighton and Hove Albion. Conceded by Raheem Sterling. Goal.  Brighton and Hove Albion 1, Manchester City 0. Glenn Murray (Brighton and Hove Albion) header from the left side of the six yard box to the bottom left corner. Assisted by Pascal Groß with a cross following a corner

## Resultados

__Resúmenes__

In [7]:
pd_df_sum = pd.read_csv(summaries_path)

In [8]:
pd_df_sum

,json_file,url,summary,article_sentences,article_sentences_ix,events_mapping
0,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,['Mario Balotelli scored his first Serie A goa...,[ 0 10 0 2 1 6 6 6 6 6 1 1 6 0 2 ...,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16, 1..."
1,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49873763,Fiorentina condemned AC Milan to a third conse...,['Fiorentina condemned AC Milan to a third con...,[0 3 2 5 2 2 0 0 2 2 2 1 1 1 3 0 2 5 1 1 1 5 5...,"[2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, ..."
2,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49865134,Cristiano Ronaldo returned from injury with a ...,['Cristiano Ronaldo returned from injury with ...,[0 0 0 0 3 0 3 1 2 0 0 3 3 3 0 0 3 0 0 0 0 0 2...,"[5, 6, 7, 8, 11, 15, 16, 17, 19, 21, 23, 26, 2..."
3,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49866723,Alexis Sanchez scored and was sent off on his ...,['Alexis Sanchez scored and was sent off on hi...,[4 0 0 1 1 0 1 0 0 1 0 0 1 0 3 0 0 4 0 0 1 1 1...,"[3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18,..."
4,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49832309,On-loan Manchester United defender Chris Small...,['On-loan Manchester United defender Chris Sma...,[4 4 4 4 4 3 4 4 2 4 4 3 3 0 4 3 4 4 4 4 4 4 4...,"[2, 4, 9, 12, 14, 15, 16, 17, 18, 20, 21, 22, ..."
...,...,...,...,...,...,...
4371,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,After lacking the final product for much of 11...,['After lacking the final product for much of ...,[1 1 4 1 1 3 1 3 3 8 0 3 1 3 1 1 8 1 2 1 1 1 1...,"[2, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18..."
4372,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,Jamiro Monteiro scored his second MLS goal on ...,['Jamiro Monteiro scored his second MLS goal\x...,[1 1 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 1 1...,"[2, 5, 7, 9, 10, 11, 13, 14, 16, 17, 22, 23, 2..."
4373,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,Nelson Bonilla's goal in first-half stoppage t...,"[""Nelson Bonilla's goal in first-half stoppage...",[ 6 0 6 6 20 6 7 7 0 6 20 9 6 8 0 ...,"[2, 4, 6, 9, 12, 13, 14, 16, 17, 18, 19, 21, 2..."
4374,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,Alphonso Davies quick-kicked the ball to Jonat...,"[""Haiti came back from a two-goal halftime def...",[ 9 9 10 8 3 10 9 4 3 9 6 3 3 3 6 ...,"[2, 3, 6, 9, 11, 12, 13, 14, 15, 16, 17, 18, 1..."


__Relación evento-frase__

In [9]:
pd_df_map = pd.read_csv(map_path)

In [10]:
pd_df_map

,event,article_sentence
0,Attempt blocked. Mario Balotelli (Brescia) rig...,Mario Balotelli scored his first Serie A goal ...
1,"Corner, Napoli. Conceded by Stefano Sabelli.",Watford striker Stefano Okaka scored the winne...
2,Attempt missed. Mario Balotelli (Brescia) left...,Mario Balotelli scored his first Serie A goal ...
3,Attempt saved. Mario Balotelli (Brescia) right...,Brescia rallied after the break but Sandro Ton...
4,"Goal! Napoli 1, Brescia 0. Dries Mertens (Nap...",Dries Mertens opened the scoring with his four...
...,...,...
111401,Attempt saved. Rodolfo Zelaya (Los Angeles Foo...,"The Rapids' lone goal came on a corner kick, a..."
111402,Attempt blocked. Keegan Rosenberry (Colorado R...,The Colorado Rapids are on a roll.
111403,Attempt missed. Rodolfo Zelaya (Los Angeles Fo...,"The Rapids' lone goal came on a corner kick, a..."
111404,Attempt blocked. Mohamed El-Munir (Los Angeles...,The Colorado Rapids are on a roll.


In [71]:
pd_df_map.iloc[89985]['event']

'Goal!  Real Sociedad 1, Barcelona 0. Aritz Elustondo (Real Sociedad) left footed shot from the left side of the box to the bottom left corner following a set piece situation.'

In [72]:
pd_df_map.iloc[89985]['article_sentence']

'The champions fell behind following a thumping shot from Aritz Elustondo in the 12th minute from a set piece.'

In [37]:
pd_df_goal = pd_df_map[pd_df_map['event'].str.contains('goal') &
                       ~pd_df_map['event'].str.contains('attempt') &
                       ~pd_df_map['event'].str.contains('Attempt')]  

In [35]:
len(pd_df_goal)

1044

In [44]:
pd_df_goal.iloc[600]['event']

'Goal!  Watford 0, Chelsea 2. Christian Pulisic (Chelsea) right footed shot from very close range to the centre of the goal. Assisted by Tammy Abraham.'

In [45]:
pd_df_goal.iloc[600]['article_sentence']

"In blustery conditions, the visitors opened the scoring after five minutes when Jorginho's delicious pass allowed Tammy Abraham to lift in his 10th goal of the season.\n"

In [47]:
pd_df_goal.iloc[601]['event']

'Goal!  Newcastle United 1, Wolverhampton Wanderers 0. Jamaal Lascelles (Newcastle United) header from very close range to the high centre of the goal. Assisted by Federico Fernández with a cross.'

In [48]:
pd_df_goal.iloc[601]['article_sentence']

"Magpies captain Jamaal Lascelles scored the opener with a powerful near-post header from Federico Fernandez's cross to light up what had been a largely dour first half.\n"